
**`ParentDocumentRetriever`의 역할**


한 권의 책을 각 장이나 절 단위로 나누고, 각 장을 따로 요약하는 것

분할 된 문서(chunks)를 인덱싱해 검색하지만, 실제 검색 결과를 반환할 때는 
이 분할 된 문서(chunks)가 속한 전체 원문 문서(Parent Document)를 함께 반환하는 방식의 검색 시스템입니다.

이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있게 됩니다.

Parent Document Retriever는 문서를 잘게 나눈 후 벡터 저장소로 검색한 다음,
해당 조각이 속한 "부모 문서"를 docstore에서 가져와서 전체 문맥을 제공합니다.


**정리**

- **문서 간의 계층 구조 활용**: `ParentDocumentRetriever`는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
- **검색 성능 향상**: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다.
  문서를 검색할 때 자주 발생하는 두 가지 상충되는 요구 사항이 있습니다:


여러 개의 텍스트 파일을 로드하기 위해 `TextLoader` 객체를 생성하고 데이터를 로드합니다.


In [ ]:
from dotenv import load_dotenv                                      # API 키 정보 로드
from langchain.storage import InMemoryStore                         # 원본(Parent) 문서를 저장할 임시 저장소
from langchain_community.document_loaders import TextLoader         # 텍스트 문서(.txt 등)를 불러오는 로더
from langchain_chroma import Chroma                                 # 벡터 저장소로 사용할 Chroma
from langchain_openai import OpenAIEmbeddings                       # OpenAI 임베딩 모델 (텍스트를 벡터화)
from langchain_text_splitters import RecursiveCharacterTextSplitter # 텍스트 분할기 (문맥을 고려한 자르기)
from langchain.retrievers import ParentDocumentRetriever            # Parent 문서를 기반으로 검색하는 리트리버

In [10]:
loaders = [
    # 파일을 로드합니다.
    TextLoader("./data/navercloud.txt",encoding="utf8"),
]

docs = [] # 문서들을 저장할 리스트 초기화
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

print(docs)

[Document(metadata={'source': './data/navercloud.txt'}, page_content='1. CLOVA Studio\n정의:CLOVA Studio는 네이버클라우드가 제공하는 AI 모델 개발 및 활용 플랫폼으로, 자연어 처리(NLP) 기반의 대규모 언어모델을 다양한 비즈니스에 쉽게 적용할 수 있게 지원합니다.\n예시:기업이 자체 고객상담 데이터를 활용해 FAQ 챗봇을 구축하거나, 보고서 자동 요약 기능을 개발할 수 있습니다.\n연관키워드:GPT, 자연어처리, 생성형 AI, 프롬프트 엔지니어링, API\n\n2. Object Storage\n정의:Object Storage는 이미지, 동영상, 로그, 백업 파일 등 비정형 데이터를 저장하고 관리할 수 있는 대용량 클라우드 스토리지 서비스입니다.\n예시:게임사가 사용자 리플레이 영상 데이터를 저장하거나, 기업이 CCTV 백업 데이터를 안전하게 장기 저장하는 데 사용합니다.\n연관키워드:비정형 데이터, 클라우드 스토리지, 백업, CDN, 데이터 보존\n\n3. Cloud DB for MySQL\n정의:Cloud DB for MySQL은 고가용성과 자동화 기능을 갖춘 네이버클라우드의 관리형 데이터베이스 서비스입니다. 백업, 복제, 모니터링 등을 자동으로 수행합니다.\n예시:쇼핑몰 플랫폼이 상품 정보와 고객 데이터를 안정적으로 관리하기 위해 Cloud DB를 사용합니다.\n연관키워드:RDS, MySQL, 데이터베이스 관리, 고가용성, 자동 백업\n\n4. AI Workbench\n정의:AI Workbench는 데이터 분석가와 머신러닝 엔지니어가 클라우드 환경에서 데이터 전처리, 모델 학습, 실험을 통합적으로 수행할 수 있는 개발환경입니다.\n예시:헬스케어 기업이 환자 데이터를 기반으로 예측 모델을 개발하기 위해 Jupyter 기반 환경에서 AI Workbench를 활용합니다.\n연관키워드:머신러닝, Jupyter, AutoML, 데이터 분석, GPU\n\n\n5. API Gateway\

## 전체 문서 검색

이 모드에서는 전체 문서를 검색하고자 합니다. 따라서 `child_splitter` 만 지정하도록 하겠습니다.

- 나중에는 `parent_splitter` 도 지정하여 결과를 비교해 보겠습니다.


In [ ]:
# 자식 문서 분할기 생성
# 원본 문서를 200자 단위로 잘라서 chunk(자식 문서)로 만듭니다.
# 이 chunk들은 벡터화되어 vectorstore에 저장됩니다.

child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# 벡터 저장소 생성 (Chroma 사용)
# embedding_function은 OpenAIEmbeddings로 설정되어,
# 텍스트 → 벡터 변환 시 OpenAI 임베딩 모델을 사용합니다.
# collection_name은 저장소 이름으로, 동일 이름일 경우 같은 컬렉션으로 취급됩니다.

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

# 원본(Parent) 문서를 저장할 저장소 생성
# InMemoryStore는 LangChain이 제공하는 메모리 기반 저장소로, 휘발성입니다.

store = InMemoryStore()

# ParentDocumentRetriever 생성
# - vectorstore: 자식 문서(분할된 chunk)가 저장되고 검색되는 벡터 저장소
# - docstore: 원본 문서(Parent 문서)를 저장하는 저장소
# - child_splitter: 문서를 분할하여 자식 문서를 생성하는 도구

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,       # 각 chunk를 → 벡터화해서 저장
    docstore=store,                # chunk가 속한 원본 문서도 저장
    child_splitter=child_splitter, # 문서 하나를 → chunk 여러 개로 나눔
)


`retriever.add_documents(docs, ids=None)` 함수로 문서목록을 추가합니다.

- `ids` 가 `None` 이면 자동으로 생성됩니다.
- `add_to_docstore=False` 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 `ids` 값이 필수 값으로 요구됩니다.


In [56]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.

retriever.add_documents(docs, ids=None, add_to_docstore=True)

# add_to_docstore=True면 docstore(예: InMemoryStore)에 저장합니다.
# 이때 자동으로 UUID 기반 doc_id가 부여됩니다 (또는 ids 지정 가능).

# ParentDocumentRetriever는 chunk로 검색하고, 원본 문서(Parent)를 반환합니다.
# 따라서 add_to_docstore=True가 아니면 원본 문서를 찾을 수 없습니다.

이 코드는 두 개의 키를 반환해야 합니다. 그 이유는 우리가 두 개의 문서를 추가했기 때문입니다.


- `store` 객체의 `yield_keys()` 메서드를 호출하여 반환된 키(key) 값들을 리스트로 변환합니다.


In [57]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['ac130422-92f8-44ac-8f6a-abc233cba210',
 '49b59266-1f1d-4a20-8561-48295470c1ce']

이제 벡터 스토어 검색 기능을 호출해 보겠습니다.

우리가 작은 청크(chunk)들을 저장하고 있기 때문에, 검색 결과로 작은 청크들이 반환되는 것을 확인할 수 있을 것입니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [58]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Cloud Functions")

len(sub_docs)

4

`sub_docs[0].page_content`를 출력합니다.


In [59]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

8. Cloud Functions
정의:Cloud Functions는 이벤트 기반으로 코드만 업로드하면 서버 없이 실행할 수 있는 FaaS(Function as a Service) 플랫폼입니다.
예시:사용자가 이미지를 업로드하면 자동으로 썸네일을 생성하는 로직을 Cloud Functions로 실행할 수 있습니다.


이제 전체 retriever에서 검색해 보겠습니다. 이 과정에서는 작은 청크(chunk)들이 위치한 **문서를 반환** 하기 때문에 상대적으로 큰 문서들이 반환될 것입니다.


`retriever` 객체의 `invoke()` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [48]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Cloud Functions")

검색된 문서(`retrieved_docs[0]`)의 일부 내용을 출력합니다.


In [55]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[1000:1500])

문서의 길이: 4882


서비스가 있는 쇼핑몰에서 API Gateway를 통해 외부 요청을 일관성 있게 처리합니다.
연관키워드:API 관리, 인증, 라우팅, Throttling, 백엔드 통합

6. Auto Scaling
정의:Auto Scaling은 서버 부하에 따라 인스턴스 수를 자동으로 조절해 서비스의 안정성과 비용 효율성을 높여주는 기능입니다.
예시:이벤트 시즌에 접속자가 급증하는 티켓 예매 사이트가 자동으로 서버를 확장하여 트래픽을 감당합니다.
연관키워드:스케일 아웃, 트래픽 관리, 탄력성, 서버 자동화

7. Cloud Insight
정의:Cloud Insight는 네이버클라우드 인프라의 모니터링과 알림 기능을 제공하는 통합 관제 도구입니다.
예시:서버의 CPU 사용률이 급증하면 알림을 받아 즉시 대응할 수 있도록 구성할 수 있습니다.
연관키워드:모니터링, 경고, 성능 추적, 대시보드

8. Cloud Functions
정의:Cloud Functions는 이벤트 기반으로 코드만 업로드하면 서버 없


## 더 큰 Chunk 의 크기를 조절

이전의 결과처럼 **전체 문서가 너무 커서 있는 그대로 검색하기에는 부적합** 할 수 있습니다.

이런 경우, 실제로 우리가 하고 싶은 것은 먼저 원시 문서를 더 큰 청크로 분할한 다음, 더 작은 청크로 분할하는 것입니다.

그런 다음 작은 청크들을 인덱싱하지만, 검색 시에는 더 큰 청크를 검색합니다 (그러나 여전히 전체 문서는 아닙니다).


- `RecursiveCharacterTextSplitter`를 사용하여 부모 문서와 자식 문서를 생성합니다.
  - 부모 문서는 `chunk_size`가 1000으로 설정되어 있습니다.
  - 자식 문서는 `chunk_size`가 200으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성됩니다.


In [61]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

`ParentDocumentRetriever`를 초기화하는 코드입니다.

- `vectorstore` 매개변수는 문서 벡터를 저장하는 벡터 저장소를 지정합니다.
- `docstore` 매개변수는 문서 데이터를 저장하는 문서 저장소를 지정합니다.
- `child_splitter` 매개변수는 하위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.
- `parent_splitter` 매개변수는 상위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.

`ParentDocumentRetriever`는 계층적 문서 구조를 처리하며, 상위 문서와 하위 문서를 별도로 분할하고 저장합니다. 이를 통해 검색 시 상위 문서와 하위 문서를 효과적으로 활용할 수 있습니다.


In [62]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

`retriever` 객체에 `docs`를 추가합니다. `retriever`가 검색할 수 있는 문서 집합에 새로운 문서들을 추가하는 역할을 합니다.


In [63]:
retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.

이제 문서의 수가 훨씬 더 많아진 것을 볼 수 있습니다. 이는 더 큰 청크(chunk)들입니다.


In [64]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환합니다.
len(list(store.yield_keys()))

6

기본 벡터 저장소가 여전히 작은 청크를 검색하는지 확인해 보겠습니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [65]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Cloud Functions")
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

8. Cloud Functions
정의:Cloud Functions는 이벤트 기반으로 코드만 업로드하면 서버 없이 실행할 수 있는 FaaS(Function as a Service) 플랫폼입니다.
예시:사용자가 이미지를 업로드하면 자동으로 썸네일을 생성하는 로직을 Cloud Functions로 실행할 수 있습니다.


이번에는 `retriever` 객체의 `invoke()` 메서드를 사용하여 문서를 검색합니다.


In [67]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Cloud Functions")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

5. API Gateway
정의:API Gateway는 여러 서비스에서 제공하는 API를 통합 관리하고 보안, 인증, 트래픽 제어 등의 기능을 제공하는 서비스입니다.
예시:여러 개의 백엔드 서비스가 있는 쇼핑몰에서 API Gateway를 통해 외부 요청을 일관성 있게 처리합니다.
연관키워드:API 관리, 인증, 라우팅, Throttling, 백엔드 통합

6. Auto Scaling
정의:Auto Scaling은 서버 부하에 따라 인스턴스 수를 자동으로 조절해 서비스의 안정성과 비용 효율성을 높여주는 기능입니다.
예시:이벤트 시즌에 접속자가 급증하는 티켓 예매 사이트가 자동으로 서버를 확장하여 트래픽을 감당합니다.
연관키워드:스케일 아웃, 트래픽 관리, 탄력성, 서버 자동화

7. Cloud Insight
정의:Cloud Insight는 네이버클라우드 인프라의 모니터링과 알림 기능을 제공하는 통합 관제 도구입니다.
예시:서버의 CPU 사용률이 급증하면 알림을 받아 즉시 대응할 수 있도록 구성할 수 있습니다.
연관키워드:모니터링, 경고, 성능 추적, 대시보드

8. Cloud Functions
정의:Cloud Functions는 이벤트 기반으로 코드만 업로드하면 서버 없이 실행할 수 있는 FaaS(Function as a Service) 플랫폼입니다.
예시:사용자가 이미지를 업로드하면 자동으로 썸네일을 생성하는 로직을 Cloud Functions로 실행할 수 있습니다.
연관키워드:서버리스, 이벤트 트리거, FaaS, 자동 실행

9. Cloud Hadoop
정의:Cloud Hadoop은 빅데이터 처리를 위한 분산 컴퓨팅 플랫폼으로, 대용량 데이터를 병렬 처리하는 데 최적화되어 있습니다.
예시:마케팅 데이터 분석을 위해 수 TB 규모의 로그 데이터를 MapReduce로 처리합니다.
연관키워드:빅데이터, MapReduce, 분산 처리, Spark
